<a href="https://colab.research.google.com/github/june12mayank/Machine_learning_notebooks/blob/master/classifier%20models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
from scipy import stats
from scipy.stats import norm, skew
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
sample=pd.read_csv('/content/SAMPLE.csv')
test=pd.read_csv('/content/TEST.csv')
train=pd.read_csv('/content/TRAINING.csv')

In [5]:
train_id=train['id']
test_id=test['id']
ntrain=train.shape[0]
ntest=test.shape[0]
y_train=train.Grade.values
df=pd.concat((train,test)).reset_index(drop=True)
df.drop(['Grade'],axis=1,inplace=True)
all_data_na = (df.isnull().sum() / len(df)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
print(missing_data)

             Missing Ratio
roof             19.477619
Roof(Area)       19.477619
Nbedrooms         0.038839
Troom             0.029129
Nbwashrooms       0.029129
Lawn(Area)        0.029129
API               0.019419
Twashrooms        0.009710


In [0]:
df['roof']=df['roof'].fillna('NO')
df['Roof(Area)']=df['Roof(Area)'].fillna(0.0)
df["Nbedrooms"] = df['Nbedrooms'].fillna(df['Nbedrooms'].mean())
df['Troom']=df['Troom'].fillna(df['Troom'].mean())
df['Nbwashrooms']=df['Nbwashrooms'].fillna(df['Nbwashrooms'].mean())
df['Lawn(Area)']=df['Lawn(Area)'].fillna(df['Lawn(Area)'].mean())
df['API']=df['API'].fillna(df['API'].mean())
df['Twashrooms']=df['Twashrooms'].fillna(df['Twashrooms'].mean())
float_cols=['API','Lawn(Area)','Nbedrooms','Nbwashrooms','Roof(Area)','Troom','Twashrooms']
df[float_cols]=df[float_cols].applymap(np.int64)
df['roof']= df['roof'].map({'NO':0,'no':0,'YES':1,'yes':1})
df.Price=df.Price.str.rstrip('$').apply(np.int64)

In [7]:
y_train

array(['D', 'B', 'C', ..., 'D', 'B', 'C'], dtype=object)

In [8]:
y_train

array(['D', 'B', 'C', ..., 'D', 'B', 'C'], dtype=object)

In [0]:
my_dict = {'A':5, 'B':4, 'C':3, 'D':2,'E':1}
y_train=np.vectorize(my_dict.get)(y_train)

In [0]:
#lbl=LabelEncoder()
#lbl.fit(list(y_train))
#y_train = lbl.transform(list(y_train))
#label transform has ambiguity in more than 2 classes and one hot encoding creates multiple cols.

In [11]:
uniq,counts=np.unique(y_train,return_counts=True)
uniq,counts

(array([1, 2, 3, 4, 5]), array([  26, 2129, 2928, 1475,  442]))

In [12]:
y_train.dtype

dtype('int64')

In [0]:
train = df[:ntrain]
test = df[ntrain:]
test_id=test['id']
test=test.drop('id',axis=1)
train=train.drop('id',axis=1)

In [0]:
#logistic Regression
from sklearn.linear_model import LogisticRegression
model2=LogisticRegression()
model2.fit(train,y_train)
pred2=model2.predict(test)

In [0]:
from sklearn.ensemble import RandomForestClassifier
model3=RandomForestClassifier(n_estimators=100,random_state=0)
model3.fit(train,y_train)
pred3=model3.predict(test)

In [0]:
model1=xgb.XGBClassifier()
model1.fit(train,y_train)
pred1=model1.predict(test)

In [0]:
from lightgbm import LGBMClassifier
model4=LGBMClassifier()
model4.fit(train,y_train)
pred4=model4.predict(test)

In [0]:
import lightgbm as lgb
lgb_params = {
    'learning_rate': 0.002,
    'max_depth': 10,
    'num_leaves': 10, 
    'metric':'acc',
    'sub_feature': 0.5,
    'min_data': 50,
    'boosting_type':'gbdt'}
dtrain_lgb = lgb.Dataset(train,y_train)
model5=lgb.train(lgb_params,dtrain_lgb,1000)
pred5=model5.predict(test)
pred5=pred5.astype(np.int64)

In [0]:
my_dict = {5:'A', 4:'B', 3:'C',2:'D',1:'E'}
pred1=np.vectorize(my_dict.get)(pred1)
pred2=np.vectorize(my_dict.get)(pred2)
pred3=np.vectorize(my_dict.get)(pred3)
pred4=np.vectorize(my_dict.get)(pred4)
pred5=np.vectorize(my_dict.get)(pred5)

In [0]:
sub=pd.DataFrame()
sub['id']=test_id
sub['Grade']=pred5
sub.to_csv('submission_jai_ho6.csv',index=False)